In [1]:
# Step-by-step notebook code for a complete Home Security System using Hybrid AlexNet+ResNet
# Project: Facial recognition with alert for unauthorized faces (strangers)

# --- CELL 1: Imports and Setup ---
!pip install torch torchvision opencv-python numpy matplotlib --quiet
!pip install pyttsx3
!pip install yagmail

import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import os, cv2, time
from PIL import Image
import numpy as np
import smtplib
from email.mime.text import MIMEText
from torchvision.models import AlexNet_Weights, ResNet18_Weights
import cv2, time, os
from PIL import Image
import torch
import pyttsx3
from email.mime.text import MIMEText
import smtplib




import yagmail

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
# --- CELL 2: Define Hybrid Model (Fixed for BCEWithLogitsLoss) ---
import torchvision.models as models
import torch.nn as nn
import torch

class HybridAlexResNet(nn.Module):
    def __init__(self, num_classes=1):
        super().__init__()
        # Load pretrained feature extractors
        self.alex = models.alexnet(weights=models.AlexNet_Weights.DEFAULT).features
        self.res = nn.Sequential(
            *list(models.resnet18(weights=models.ResNet18_Weights.DEFAULT).children())[:-1]
        )

        # Combined feature vector size = 256*6*6 (AlexNet) + 512 (ResNet)
        self.classifier = nn.Sequential(
            nn.Linear(256*6*6 + 512, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
            # ❌ Removed Sigmoid
        )

    def forward(self, x):
        a = torch.flatten(self.alex(x), 1)
        r = torch.flatten(self.res(x), 1)
        x = torch.cat((a, r), dim=1)
        return self.classifier(x)  # Raw logits


In [3]:
# --- CELL 3: Dataset Class ---
# --- CELL 3: Dataset Class ---
from torch.utils.data import Dataset
from PIL import Image
import torch

class FaceDataset(Dataset):
    def __init__(self, image_paths, labels, transform):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert('RGB')
        image = self.transform(image)
        return image, torch.tensor(self.labels[idx], dtype=torch.float32)


In [4]:
# --- CELL 4: Load Dataset ---
 # Only authorized faces stored here
# --- CELL 4: Load Dataset (Improved for nested folder) ---
import os
from torchvision import transforms

authorized_dir = 'faces/authorized'
strangers_dir = 'faces/strangers'
image_paths = []
labels = []

# Walk through each subfolder inside 'authorized'
for root, dirs, files in os.walk(authorized_dir):
    for file in files:
        if file.lower().endswith(('.jpg', '.jpeg', '.png')):
            image_paths.append(os.path.join(root, file))
            labels.append(1)  # All are authorized# Load stranger images
            
for img in os.listdir(strangers_dir):
    if img.endswith(('.jpg', '.png')):
        image_paths.append(os.path.join(strangers_dir, img))
        labels.append(0)  # Stranger
            

print("Found images:", len(image_paths))
print("Sample image paths:", image_paths[:3])

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor()
])

dataset = FaceDataset(image_paths, labels, transform)
loader = DataLoader(dataset, batch_size=4, shuffle=True)


Found images: 21
Sample image paths: ['faces/authorized/user2/2025-05-03-141335.jpg', 'faces/authorized/user2/2025-05-03-141328.jpg', 'faces/authorized/user1/2025-05-01-223920.jpg']


In [5]:
# --- CELL 5: Train the Model (Fixed) ---

# Make sure your HybridAlexResNet DOES NOT include Sigmoid in the final layer if you're using BCEWithLogitsLoss

model = HybridAlexResNet()  # Your model must return raw logits (no sigmoid)
criterion = nn.BCEWithLogitsLoss()  # More numerically stable than BCE + Sigmoid

optimizer = torch.optim.Adam(
    model.parameters(), lr=1e-4, weight_decay=1e-4  # L2 regularization
)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, factor=0.5, patience=2  # Reduce LR if val_loss plateaus
)

# Split dataset into 80% train and 20% validation
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_ds, val_ds = torch.utils.data.random_split(dataset, [train_size, val_size])

# Use num_workers=0 if you're on Windows or face issues; keep pin_memory=False if not using GPU
train_loader = DataLoader(train_ds, batch_size=8, shuffle=True, num_workers=0, pin_memory=False)
val_loader   = DataLoader(val_ds, batch_size=8, shuffle=False, num_workers=0, pin_memory=False)

best_val_loss = float('inf')
num_epochs = 1

for epoch in range(1, num_epochs + 1):
    # ---- Training Phase ----
    model.train()
    train_loss = 0.0
    for images, labels in train_loader:
        outputs = model(images).view(-1)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss /= len(train_loader)

    # Show learning rate
    for param_group in optimizer.param_groups:
        print(f"[Epoch {epoch}] Current LR: {param_group['lr']}")

    # ---- Validation Phase ----
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for images, labels in val_loader:
            outputs = model(images).view(-1)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    val_loss /= len(val_loader)

    # Step the scheduler
    scheduler.step(val_loss)

    # Save best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "best_hybrid_model.pth")

    print(f"Epoch {epoch}/{num_epochs} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

print("✅ Training complete. Best validation loss:", best_val_loss)


[Epoch 1] Current LR: 0.0001
Epoch 1/1 | Train Loss: 0.4689 | Val Loss: 0.2473
✅ Training complete. Best validation loss: 0.2472943812608719


In [6]:
# --- CELL 6: Load Model for Inference ---
model.load_state_dict(torch.load("hybrid_model.pth" , map_location=torch.device('cpu')))
model.eval()

HybridAlexResNet(
  (alex): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (res): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_ru

In [ ]:
import os
import cv2
import time
import torch
import pyttsx3
import yagmail
from PIL import Image
from torchvision import transforms

# --- Voice Engine ---
engine = pyttsx3.init()
engine.setProperty('rate', 150)  # Adjust speech rate
engine.setProperty('volume', 1.0)  # Max volume

def speak(text):
    engine.say(text)
    engine.runAndWait()

# --- Transformation for Face Image ---
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# --- Load Pre-trained Model ---
# Ensure your model is loaded appropriately
# Example:
# model = torch.load('your_model.pth')
# model.eval()

# --- Email Alert Function using Yagmail with OAuth2 ---
def send_email_alert():
    try:
        yag = yagmail.SMTP('agrawalshubhangi03@gmail.com', oauth2_file='credentials.json')
        yag.send(
            to='agrawalshubhangi03@gmail.com',
            subject='Home Security Alert',
            contents='Stranger detected by your home security system after 5 failed attempts.'
        )
        print("Alert email sent successfully!")
    except Exception as e:
        print("Failed to send email:", e)

# --- Face Detection & Webcam ---
cap = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
frame_count = 0
stranger_count = 0
cooldown = False
cooldown_start_time = None

while True:
    ret, frame = cap.read()
    frame_count += 1
    if not ret or frame_count % 5 != 0:
        continue  # Skip frames for better speed

    if cooldown:
        elapsed_time = time.time() - cooldown_start_time
        if elapsed_time < 10:
            remaining_time = int(10 - elapsed_time)
            print(f"Cooldown active. Please wait {remaining_time} seconds.")
            speak(f"Cooldown active. Please wait {remaining_time} seconds.")
            continue
        else:
            cooldown = False
            stranger_count = 0

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        face_img = frame[y:y+h, x:x+w]
        face_pil = Image.fromarray(cv2.resize(face_img, (224,224))).convert('RGB')
        face_tensor = transform(face_pil).unsqueeze(0)

        with torch.no_grad():
            pred = model(face_tensor).item()

        print(f"Prediction Score: {pred:.4f}")

        if pred >= 0.7:
            label = 'Authorized'
            color = (0,255,0)
            stranger_count = 0  # Reset counter on successful recognition
            speak("Authentication granted.")
        else:
            label = 'Stranger'
            color = (0,0,255)
            stranger_count += 1
            speak(f"Authentication not granted. Attempt {stranger_count} of 5.")
            ts = int(time.time())
            if not os.path.exists("strangers"):
                os.makedirs("strangers")
            filename = f'strangers/stranger_{ts}.jpg'
            cv2.imwrite(filename, face_img)
            print(f"🖼️ Stranger snapshot saved at {filename}")

            if stranger_count >= 5:
                send_email_alert()
                speak("Maximum unauthorized attempts reached. Alert email sent.")
                cooldown = True
                cooldown_start_time = time.time()

        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    cv2.imshow("Security Feed", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Prediction Score: -4.5904
🖼️ Stranger snapshot saved at strangers/stranger_1747198199.jpg
Prediction Score: -5.2973
🖼️ Stranger snapshot saved at strangers/stranger_1747198206.jpg
Prediction Score: -1.0397
🖼️ Stranger snapshot saved at strangers/stranger_1747198212.jpg
Prediction Score: -5.0423
🖼️ Stranger snapshot saved at strangers/stranger_1747198217.jpg
Prediction Score: -3.5129
🖼️ Stranger snapshot saved at strangers/stranger_1747198219.jpg
Failed to send email: HTTP Error 400: Bad Request
Cooldown active. Please wait 9 seconds.
Cooldown active. Please wait 8 seconds.
Cooldown active. Please wait 3 seconds.
Cooldown active. Please wait 2 seconds.
Prediction Score: 0.9567
Prediction Score: 1.8321
Prediction Score: 1.0866
Prediction Score: 1.6843
Prediction Score: 1.6080
Prediction Score: 2.0221
Prediction Score: 1.7000
Prediction Score: 0.9590
Prediction Score: 1.3882
Prediction Score: 1.5386
Prediction Score: 0.0424
🖼️ Stranger snapshot saved at strangers/stranger_1747198268.jpg
P

aplay: pcm_write:2127: write error: Interrupted system call
